In [1]:
# Program places orders

# STATUS: Incomplete

#***          Start ib_insync (run once)       *****
#___________________________________________________

from ib_insync import *
util.startLoop()
ib=IB().connect('127.0.0.1', 7496, clientId=1) # kavi TWS live
# ib=IB().connect('127.0.0.1', 4001, clientId=1) # kavi IBG live

In [9]:
import pandas as pd
import datetime

#******         Paths and variables         ****
#_______________________________________________

datapath = r'./zdata/'
base_rom = 10    # 90% RoM atleast

df_orders = pd.read_pickle(datapath+'df_orders.pkl')

# Filter for puts below strike threshold
df_orders = df_orders[df_orders.Strike <= df_orders.Threshold]

# Filter out the top Return on Margin trades
df_orders1 = df_orders[df_orders.RoM >= base_rom].reset_index(drop=True)
df_orders1 = df_orders1.sort_values('RoM', ascending=False)

# Compute margin for the trade
df_orders1['Margin'] = df_orders1.marginpct*df_orders1.Mlot*df_orders1.Strike

In [10]:
# Filter only the trades expiring in this month
# df = df_orders1[df_orders1.DTE == df_orders1.DTE.min()].reset_index(drop=True)
df = df_orders1

In [11]:
# sorted(df_orders1.ibSymbol.unique())
len(df)

115

In [12]:
# total margin
df.Margin.sum()

8360520.0255

In [13]:
# Sort by Symbol for easy reference in Excel
df.sort_values(by='ibSymbol').to_excel(datapath+'temp.xlsx', index=None, freeze_panes=(1,1))

In [14]:
#------------------------------------------------
#####    WARNING: THIS PLACES THE TRADE  #######
#________________________________________________

limitTrade = [ib.placeOrder(contract, order) 
              for contract, order in zip(df.Contract, df.Orders)]

ERROR:ib_insync.wrapper:Error 388, reqId 2822: Order size 4,500 is smaller than the minimum required size of 7,000.
ERROR:ib_insync.wrapper:Error 388, reqId 2821: Order size 8,500 is smaller than the minimum required size of 13,000.
ERROR:ib_insync.wrapper:Error 388, reqId 2878: Order size 8,500 is smaller than the minimum required size of 13,000.
ERROR:ib_insync.wrapper:Error 388, reqId 2820: Order size 34,000 is smaller than the minimum required size of 55,000.
ERROR:ib_insync.wrapper:Error 388, reqId 2823: Order size 4,500 is smaller than the minimum required size of 4,700.
ERROR:ib_insync.wrapper:Error 388, reqId 2826: Order size 7,000 is smaller than the minimum required size of 12,000.
ERROR:ib_insync.wrapper:Error 388, reqId 2828: Order size 7,000 is smaller than the minimum required size of 12,000.
ERROR:ib_insync.wrapper:Error 388, reqId 2835: Order size 13,000 is smaller than the minimum required size of 16,000.
ERROR:ib_insync.wrapper:Error 388, reqId 2856: Order size 13,000

ERROR:ib_insync.wrapper:Error 388, reqId 2877: Order size 13,000 is smaller than the minimum required size of 16,000.
ERROR:ib_insync.wrapper:Error 388, reqId 2837: Order size 2,800 is smaller than the minimum required size of 3,800.
ERROR:ib_insync.wrapper:Error 388, reqId 2843: Order size 1,700 is smaller than the minimum required size of 2,000.
ERROR:ib_insync.wrapper:Error 388, reqId 2849: Order size 5,500 is smaller than the minimum required size of 7,000.
ERROR:ib_insync.wrapper:Error 388, reqId 2912: Order size 5,500 is smaller than the minimum required size of 7,000.
ERROR:ib_insync.wrapper:Error 388, reqId 2854: Order size 28,000 is smaller than the minimum required size of 34,000.
ERROR:ib_insync.wrapper:Error 388, reqId 2855: Order size 2,400 is smaller than the minimum required size of 2,850.
ERROR:ib_insync.wrapper:Error 388, reqId 2868: Order size 2,400 is smaller than the minimum required size of 2,850.
ERROR:ib_insync.wrapper:Error 388, reqId 2873: Order size 2,400 is s

ERROR:ib_insync.wrapper:Error 388, reqId 2899: Order size 2,400 is smaller than the minimum required size of 2,850.
ERROR:ib_insync.wrapper:Error 388, reqId 2865: Order size 4,950 is smaller than the minimum required size of 6,000.
ERROR:ib_insync.wrapper:Error 388, reqId 2924: Order size 4,950 is smaller than the minimum required size of 6,000.
ERROR:ib_insync.wrapper:Error 388, reqId 2869: Order size 11,000 is smaller than the minimum required size of 12,000.
ERROR:ib_insync.wrapper:Error 388, reqId 2875: Order size 9,000 is smaller than the minimum required size of 15,000.
ERROR:ib_insync.wrapper:Error 388, reqId 2880: Order size 6,000 is smaller than the minimum required size of 8,000.
ERROR:ib_insync.wrapper:Error 388, reqId 2900: Order size 6,000 is smaller than the minimum required size of 8,000.
ERROR:ib_insync.wrapper:Error 388, reqId 2892: Order size 3,500 is smaller than the minimum required size of 4,100.
ERROR:ib_insync.wrapper:Error 388, reqId 2896: Order size 500 is smal

ERROR:ib_insync.wrapper:Error 388, reqId 2903: Order size 7,000 is smaller than the minimum required size of 12,000.
ERROR:ib_insync.wrapper:Error 388, reqId 2904: Order size 7,000 is smaller than the minimum required size of 12,000.
ERROR:ib_insync.wrapper:Error 388, reqId 2905: Order size 7,000 is smaller than the minimum required size of 12,000.
ERROR:ib_insync.wrapper:Error 388, reqId 2910: Order size 4,500 is smaller than the minimum required size of 4,700.
ERROR:ib_insync.wrapper:Error 388, reqId 2920: Order size 1,700 is smaller than the minimum required size of 2,000.
ERROR:ib_insync.wrapper:Error 388, reqId 2926: Order size 2,400 is smaller than the minimum required size of 2,850.
ERROR:ib_insync.wrapper:Error 388, reqId 2927: Order size 5,500 is smaller than the minimum required size of 7,000.
ERROR:ib_insync.wrapper:Error 388, reqId 2928: Order size 2,800 is smaller than the minimum required size of 3,400.
ERROR:ib_insync.wrapper:Error 388, reqId 2929: Order size 900 is smal

#------------------------------------------------
#####         Code to cancel all trades    ############
#________________________________________________
cancel = ib.reqGlobalCancel()


In [15]:
# Put placed orders to a dataframe
df_orders_placed = pd.DataFrame(limitTrade)

In [16]:
# Write orders placed to file
x  =  datapath + datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_orders_placed.xlsx"
writer = pd.ExcelWriter(x)
df_orders_placed.to_excel(writer, 'all-options', index=False, header=None)
writer.save()

In [ ]:
ib.disconnect()

In [ ]:
list(df_orders)

In [ ]:
df.head()